Load tf-idf vectors

In [22]:
import pickle 
sparsevectors={}
with open('text_vectors.pkl', 'rb') as f:
    sparsevectors = pickle.load(f)

Load train and our edges

In [19]:
import pandas as pd

df_edges=pd.read_csv('train.csv')
df_edges_all = df_edges[['id1', 'id2']]

# load edges from tzt file (new_edges.txt)
dfouredges=pd.read_csv("new_edges.txt",header=None,names=["id1","id2"])

dfouredges["id1"]=dfouredges["id1"].astype(int)
dfouredges["id2"]=dfouredges["id2"].astype(int)

df_edges_all=pd.concat([df_edges_all, dfouredges], ignore_index=True)


Build graph

In [20]:
# The existing graph
import networkx as nx
G= nx.Graph()
#add nodes
nodes = pd.unique(df_edges_all[['id1', 'id2']].values.ravel())
G.add_nodes_from(nodes)
#add edges
edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==1].itertuples()]
G.add_edges_from(edges)
edges2 = [(row.id1,row.id2) for row in dfouredges.itertuples()]
G.add_edges_from(edges2)


# keeping all the negative edges so we can compare results
no_edges = [(row.id1,row.id2) for row in df_edges[df_edges["label"]==0].itertuples()]

Helper functions

In [21]:
import math
# Adding methods proposed in the https://link.springer.com/content/pdf/10.1038/s41598-019-57304-y.pdf
def shortest_path(G:nx.Graph,source:int,target:int)->float:
    try:
        p=nx.shortest_path(G,source=source,target=target)
        return len(p)-1
    except nx.NetworkXNoPath:
        return float("inf")

def distance(G:nx.Graph,source:int,target:int)->float:
   '''
   This method can be changed to any distance
   '''
   return shortest_path(G,source,target)

from sklearn.metrics.pairwise import cosine_similarity
def distance_description(G,source:int,target:int)->float:
    '''
    This method can be changed to any distance
    '''
    
    
    sp1=sparsevectors[source]
    sp2=sparsevectors[target]
    
    return cosine_similarity(sp1,sp2)[0][0]


def closeness_centrality(G:nx.Graph, source:int, target:int)->float:
    dxy = nx.shortest_path(G,source=source,target=target)
    return G.number_of_nodes()/shortest_path(G,source,target)

def common_neighbors(G:nx.Graph, source:int, target:int)->list:
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    return common_neighbors

def CCPA(G:nx.Graph, source:int, target:int,a:float = 0.5)->float:
    '''
    Common Neighbor and Centrality based Parameterized Algorithm
    '''
    return a*closeness_centrality(G,source,target)+(1-a)*len(common_neighbors(G,source,target))

def CND(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor and Distance
    '''
    cn = common_neighbors(G,source,target)
    if len(cn)>0:
        return (len(cn)+1)/2
    else:
        if distance(G,source,target)==0:
            return 0
        return 1/distance(G,source,target)

def PA(G:nx.Graph, source:int, target:int)->float:
    '''
    Preferential Attachment
    '''
    return G.degree[source]*G.degree[target]

def AA(G:nx.Graph, source:int, target:int)->float:
    '''
    Adamic Adar
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / math.log(degree)
    return similarity

def CN(G:nx.Graph, source:int, target:int)->float:
    '''
    Common Neighbor
    '''
    return len(common_neighbors(G,source,target))

def SI(G:nx.Graph, source:int, target:int)->float:
    '''
    Sorensen Index
    '''
    if (G.degree[source]+G.degree[target]) ==0:
        return 0
    return 2*CN(G,source,target)/(G.degree[source]+G.degree[target])

def JI(G:nx.Graph, source:int, target:int)->float:
    '''
    Jaccard Index
    '''
    s_neighbors = list(G.adj[source].keys())
    t_neighbors = list(G.adj[target].keys())
    common_neighbors = set(s_neighbors).intersection(t_neighbors)
    all_neighbors = set(s_neighbors).union(t_neighbors)
    if len(all_neighbors)==0:
        return 0
    return len(common_neighbors)/len(all_neighbors)

def RA(G:nx.Graph, source:int, target:int)->float:
    '''
    Resource Allocation
    '''
    similarity = 0.0
    for neighbor in common_neighbors(G,source,target):
        degree = G.degree[neighbor]
        if degree > 1:
            similarity += 1 / degree
    return similarity

def HPI(G:nx.Graph, source:int, target:int)->float:
    '''
    Hub Promoted Index
    '''
    if min([G.degree[source],G.degree[target]]) ==0 :
        return 0
    return CN(G,source,target)/min([G.degree[source],G.degree[target]])

def create_metric_vector(tup):
    G=tup[0]
    source= tup[1]
    target= tup[2]
    functions = [CND, PA, AA, CN, SI, JI, RA, HPI,distance_description]
    outputs = []
    for func in functions:
        output = func(G, source, target)
        outputs.append(output)
    return outputs




In [25]:
from multiprocessing import Pool
from tqdm import tqdm

def makedataset(df):
    label=df.label
    data=[]
    allind=len(df.index)
    counter=0
    tups=[]
    for i in tqdm(range(len(df.index))):
        row = df.iloc[i]

        tups.append((G,row.id1,row.id2))
    with Pool(4) as p:
        data=p.map(create_metric_vector, tups)
    
    datadf=pd.DataFrame(data)
    return datadf,label

In [ ]:
from sklearn.model_selection import train_test_split


import time

start = time.time()
X,Y=makedataset(df_edges)
end = time.time()
print(end - start)


#start = time.time()
#X,Y=makedataset(df_edges)
#end = time.time()
#print(end - start)

100%|████████████████████████████████| 948232/948232 [00:44<00:00, 21275.22it/s]


In [ ]:
from sklearn.metrics import f1_score
model = XGBClassifier()

X_train, X_test, y_trainn, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
model.fit(X_train, y_trainn)
preds = model.predict(X_test)
f1=f1_score(y_test, preds)
print(f1)